In [1]:
import numpy as np

In [2]:
import pandas as pd

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## Name | Title

In [3]:
train_test_data = [train,test]

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.',expand=False)

In [4]:
title_mapping = {'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3
                 , 'Dr':4, 'Rev':4, 'Mlle':4, 'Major':4, 'Col':4
                 ,'Countess':4, 'Capt':4, 'Ms':4, 'Sir':4, 'Lady':4
                 , 'Mme':4, 'Don':4, 'Jonkheer':4
                 }

In [5]:
train['Title'] = train['Title'].map(title_mapping)

In [6]:
title_mapping = {'Mr':0, 'Miss':1, 'Mrs':2, 'Master':3
                 , 'Col':4, 'Rev':4, 'Ms':4, 'Dr':4, 'Dona':4
                 }

test['Title'] = test['Title'].map(title_mapping)

## Sex


In [7]:
sex_mapping = {'male':0, 'female':1}

for dataset in train_test_data:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

## Hase Age

In [8]:
train.loc[train['Age'].isnull(), 'Null_Age'] = 0
test.loc[test['Age'].isnull(), 'Null_Age'] = 0

train.loc[train['Age'].notnull(), 'Null_Age'] = 1
test.loc[test['Age'].notnull(), 'Null_Age'] = 1


## Age

In [9]:
# fill missing age with median age for each title(Mr,Mrs,Miss,Master,Others)
# Age 결측치 해당 Title의 나이의 중앙값으로 채우기

train['Age'].fillna(train.groupby('Title')['Age'].transform('mean'),inplace=True)
test['Age'].fillna(test.groupby('Title')['Age'].transform('mean'),inplace=True)

In [10]:
for dataset in train_test_data:
    dataset.loc[dataset['Age'] <= 17, 'Age'] =0
    dataset.loc[(dataset['Age'] >17) & (dataset['Age'] <= 24), 'Age'] = 1
    dataset.loc[(dataset['Age'] >24) & (dataset['Age'] <= 34), 'Age'] = 2
    dataset.loc[(dataset['Age'] >34) & (dataset['Age'] <= 44), 'Age'] = 3
    dataset.loc[(dataset['Age'] >44) & (dataset['Age'] <= 60), 'Age'] = 4
    dataset.loc[dataset['Age'] >60, 'Age'] = 5

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Null_Age
0,1,0,3,"Braund, Mr. Owen Harris",0,1.0,1,0,A/5 21171,7.2500,NaN,S,0,1.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,3.0,1,0,PC 17599,71.2833,C85,C,2,1.0
2,3,1,3,"Heikkinen, Miss. Laina",1,2.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,3.0,1,0,113803,53.1000,C123,S,2,1.0
4,5,0,3,"Allen, Mr. William Henry",0,3.0,0,0,373450,8.0500,NaN,S,0,1.0


In [11]:
# for dataset in train_test_data:
#     dataset.loc[dataset['Age'] <= 10, 'Age'] =0
#     dataset.loc[(dataset['Age'] >10) & (dataset['Age'] <= 17), 'Age'] = 0.4
#     dataset.loc[(dataset['Age'] >17) & (dataset['Age'] <= 21), 'Age'] = 0.8
#     dataset.loc[(dataset['Age'] >21) & (dataset['Age'] <= 28), 'Age'] = 1.2
#     dataset.loc[(dataset['Age'] >28) & (dataset['Age'] <= 32), 'Age'] = 1.6
#     dataset.loc[(dataset['Age'] >32) & (dataset['Age'] <= 45), 'Age'] = 2
#     dataset.loc[dataset['Age'] >45, 'Age'] = 2.4

# train.head()

## Embarked

In [12]:
# 대부분 S embark 에서 탐 => fillna('S')

train['Embarked'] = train['Embarked'].fillna('S')
test['Embarked'] = test['Embarked'].fillna('S')

In [13]:
embarked_mapping = {'S':0, 'C':1, 'Q':2}
train['Embarked'] = train['Embarked'].map(embarked_mapping)
test['Embarked'] = test['Embarked'].map(embarked_mapping)

## Fare

In [14]:
test['Fare'].fillna(
    test.groupby('Pclass')['Fare'].transform('median'), inplace=True
)

In [15]:
train['Fare'].isnull().sum(), test['Fare'].isnull().sum()

(0, 0)

## group_size

In [16]:
for ticket_num in train['Ticket'].unique():
    train.loc[train['Ticket']==ticket_num,'group_size'] = len(train[train['Ticket']==ticket_num])

for ticket_num in test['Ticket'].unique():
    test.loc[test['Ticket']==ticket_num,'group_size'] = len(test[test['Ticket']==ticket_num])

In [17]:
train['Fare'] = train['Fare']/train['group_size']
test['Fare'] = test['Fare']/test['group_size']

In [18]:
train_test_data = [train,test]

for dataset in train_test_data:
    dataset.loc[dataset['group_size'] == 1, 'group_size'] = 0
    dataset.loc[dataset['group_size'] == 2, 'group_size'] = 0.4
    dataset.loc[(dataset['group_size'] == 3) | (dataset['group_size'] == 4), 'group_size'] = 0.8
    dataset.loc[dataset['group_size'] > 4, 'group_size'] = 1.2


In [19]:
train_test_data = [train,test]
for dataset in train_test_data:

    dataset.loc[dataset['Fare'] <= 7,'Fare'] = 0
    dataset.loc[(dataset['Fare'] >7) & (dataset['Fare'] <=8.8), 'Fare'] = 0.4
    dataset.loc[(dataset['Fare'] >8.8) & (dataset['Fare'] <=17), 'Fare'] = 0.8
    dataset.loc[(dataset['Fare'] >17) & (dataset['Fare'] <=30), 'Fare'] = 1.2
    dataset.loc[(dataset['Fare'] >30) & (dataset['Fare'] <=100), 'Fare'] = 1.6
    dataset.loc[dataset['Fare'] > 100,'Fare'] = 2


## Hase Cabin

In [20]:
train.loc[train['Cabin'].isnull(), 'Null_Cabin'] = 0
test.loc[test['Cabin'].isnull(), 'Null_Cabin'] = 0

train.loc[train['Cabin'].notnull(), 'Null_Cabin'] = 1
test.loc[test['Cabin'].notnull(), 'Null_Cabin'] = 1


In [21]:
train['Cabin'].isnull().sum()

687

In [22]:
train['Null_Cabin'].unique()

array([0., 1.])

## Cabin

In [23]:
train['Cabin'] = train['Cabin'].str[:1]
test['Cabin'] = test['Cabin'].str[:1]

In [24]:
cabin_mapping = {"A": 0, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}

train['Cabin'] = train['Cabin'].map(cabin_mapping)
test['Cabin'] = test['Cabin'].map(cabin_mapping)

In [25]:
# test.loc[test['Pclass']==3,'Cabin']=test.loc[test['Pclass']==3,'Cabin'].fillna(2.2)
# test.loc[test['Pclass']==3,'Cabin']=test.loc[test['Pclass']==3,'Cabin'].fillna(2.2)

In [26]:
# # fill missing Fare with median fare for each Pclass
train["Cabin"].fillna(train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
test["Cabin"].fillna(test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

In [27]:
train['Cabin'].isnull().sum()

0

## FamilySize

In [28]:
train['FamilySize'] = train['SibSp'] + train['Parch'] + 1
test['FamilySize'] = test['SibSp'] + test['Parch'] + 1

In [29]:
family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
train['FamilySize'] = train['FamilySize'].map(family_mapping)
test['FamilySize'] = test['FamilySize'].map(family_mapping)

In [30]:
# train_test_data = [train,test]
# for dataset in train_test_data:

#     dataset.loc[dataset['FamilySize'] == 1,'FamilySize'] = 0.4
#     dataset.loc[(dataset['FamilySize'] == 2), 'FamilySize'] = 0.8
#     dataset.loc[(dataset['FamilySize'] == 3), 'FamilySize'] = 1.2
#     dataset.loc[dataset['FamilySize'] > 3,'FamilySize'] = 1.6


In [31]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Title', 'Null_Age',
       'group_size', 'Null_Cabin', 'FamilySize'],
      dtype='object')

## 정규화

In [32]:
from sklearn.preprocessing import MinMaxScaler

In [33]:
columns = ['Pclass', 'Sex', 'Age', 'Fare', 'Cabin',
       'Embarked', 'Title', 'Null_Age', 'group_size', 'Null_Cabin',
       'FamilySize']
# 객체 생성
scaler = MinMaxScaler()
# 데이터 셑 변환, fit(), transform()
scaler.fit(train[columns])
scaled = scaler.transform(train[columns])

#transforma()시 스케일 변환된 데이터 세트가 ndarray로 반환돼 이를 DataFrame으로 변환
df_scaled = pd.DataFrame(data=scaled, columns=columns)

print('최솟값')             # 0에 가까워짐
print(df_scaled.min())
print('\n최댓값')
print(df_scaled.max()) 

최솟값
Pclass        0.0
Sex           0.0
Age           0.0
Fare          0.0
Cabin         0.0
Embarked      0.0
Title         0.0
Null_Age      0.0
group_size    0.0
Null_Cabin    0.0
FamilySize    0.0
dtype: float64

최댓값
Pclass        1.0
Sex           1.0
Age           1.0
Fare          1.0
Cabin         1.0
Embarked      1.0
Title         1.0
Null_Age      1.0
group_size    1.0
Null_Cabin    1.0
FamilySize    1.0
dtype: float64


In [34]:
train.loc[:,columns] = df_scaled

In [35]:
columns = ['Pclass', 'Sex', 'Age', 'Fare', 'Cabin',
       'Embarked', 'Title', 'Null_Age', 'group_size', 'Null_Cabin',
       'FamilySize']
# 객체 생성
scaler = MinMaxScaler()
# 데이터 셑 변환, fit(), transform()
scaler.fit(test[columns])
scaled = scaler.transform(test[columns])

#transforma()시 스케일 변환된 데이터 세트가 ndarray로 반환돼 이를 DataFrame으로 변환
df_scaled = pd.DataFrame(data=scaled, columns=columns)

print('최솟값')             # 0에 가까워짐
print(df_scaled.min())
print('\n최댓값')
print(df_scaled.max()) 

최솟값
Pclass        0.0
Sex           0.0
Age           0.0
Fare          0.0
Cabin         0.0
Embarked      0.0
Title         0.0
Null_Age      0.0
group_size    0.0
Null_Cabin    0.0
FamilySize    0.0
dtype: float64

최댓값
Pclass        1.0
Sex           1.0
Age           1.0
Fare          1.0
Cabin         1.0
Embarked      1.0
Title         1.0
Null_Age      1.0
group_size    1.0
Null_Cabin    1.0
FamilySize    1.0
dtype: float64


In [36]:
test.loc[:,columns] = df_scaled

---

## modeling

In [37]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [38]:
features_drop = ['Ticket','SibSp','Parch','Name']
train = train.drop(features_drop,axis=1)
test = test.drop(features_drop, axis=1)

In [39]:
train_data = train.drop('Survived',axis=1)
target = train['Survived']

train_data.shape,target.shape

((891, 12), (891,))

In [40]:
test.describe()

,PassengerId,Pclass,Sex,Age,Fare,Cabin,Embarked,Title,Null_Age,group_size,Null_Cabin,FamilySize
count,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000,418.000000
mean,1100.500000,0.632775,0.363636,0.400000,0.432057,0.702951,0.232057,0.187201,0.794258,0.106061,0.217703,0.083971
std,120.810458,0.420919,0.481622,0.248545,0.248209,0.234738,0.342758,0.254641,0.404727,0.214503,0.413179,0.151907
min,892.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,0.000000,0.000000,0.200000,0.200000,0.666667,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,1100.500000,1.000000,0.000000,0.400000,0.400000,0.833333,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,1204.750000,1.000000,1.000000,0.600000,0.600000,0.833333,0.500000,0.250000,1.000000,0.000000,0.000000,0.100000
max,1309.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [41]:
dd

NameError: name 'dd' is not defined

In [ ]:
# RandomForest
parameters = {'n_estimators':[400,500]
              ,'max_depth':[4,6,8]
              ,'min_samples_split':[2,4,6]
              ,'min_samples_leaf' : [2,4,6]
              , 'random_state':[32,49]
              }

rf_clf = RandomForestClassifier()

grid_clf = GridSearchCV(rf_clf,param_grid=parameters,scoring='accuracy',cv=5,n_jobs=-1)

grid_clf.fit(train_data,target)
print(grid_clf.best_params_)
print(grid_clf.best_score_)



In [ ]:
clf = RandomForestClassifier(n_estimators=750
                             , max_depth=5
                             #,min_samples_split=6
                             ,min_samples_leaf=15
                             , random_state=1)
clf.fit(train_data, target)

prediction = clf.predict(test)

## testing

In [42]:
submission_a = pd.read_csv('../data/submission_a.csv')
del submission_a['PassengerId']

In [ ]:
accuracy = accuracy_score(prediction, submission_a)

accuracy

0.8086124401913876

In [ ]:
acc_list=[]

for r in range(2,20):
    clf = RandomForestClassifier(n_estimators=660, max_depth=r
                                 #, min_samples_split=r
                                 , min_samples_leaf=14, random_state=1, n_jobs=-1)
    clf.fit(train_data, target) # 학습
    Y_pred = clf.predict(test) # 테스트 데이터로 예측값 추출    
    
    accuracy = accuracy_score(Y_pred, submission_a)
    print(r, accuracy)
    acc_list.append(accuracy)

acc_list.index(max(acc_list)),max(acc_list)

2 0.7822966507177034
3 0.7870813397129187
4 0.8014354066985646
5 0.8157894736842105
6 0.8038277511961722
7 0.7966507177033493
8 0.7990430622009569
9 0.7990430622009569
10 0.7942583732057417
11 0.7990430622009569
12 0.7990430622009569
13 0.7990430622009569
14 0.7990430622009569
15 0.7990430622009569
16 0.7990430622009569
17 0.7990430622009569
18 0.7990430622009569
19 0.7990430622009569


(3, 0.8157894736842105)

In [43]:
clf = RandomForestClassifier(n_estimators=660
                             , max_depth=5
                            #  ,min_samples_split=6
                             ,min_samples_leaf=14
                             , random_state=1
                             ,n_jobs=-1)
clf.fit(train_data, target)

prediction = clf.predict(test)

In [44]:
accuracy = accuracy_score(prediction, submission_a)

accuracy

0.8157894736842105

In [ ]:
# feature importance 추출

print("Feature importances:\n{0}".format(np.round(clf.feature_importances_,3)))

# feature 별 importance 매핑

for name, value in zip(train_data.columns
                       ,clf.feature_importances_):
    print('{0} : {1:.3f}'.format(name, value))

Feature importances:
[0.032 0.082 0.286 0.025 0.053 0.07  0.014 0.302 0.003 0.037 0.047 0.048]
PassengerId : 0.032
Pclass : 0.082
Sex : 0.286
Age : 0.025
Fare : 0.053
Cabin : 0.070
Embarked : 0.014
Title : 0.302
Null_Age : 0.003
group_size : 0.037
Null_Cabin : 0.047
FamilySize : 0.048


In [ ]:
# feature importance 추출

print("Feature importances:\n{0}".format(np.round(clf.feature_importances_,3)))

# feature 별 importance 매핑

for name, value in zip(train.columns
                       ,clf.feature_importances_):
    print('{0} : {1:.3f}'.format(name, value))

Feature importances:
[0.032 0.082 0.286 0.025 0.053 0.07  0.014 0.302 0.003 0.037 0.047 0.048]
PassengerId : 0.032
Survived : 0.082
Pclass : 0.286
Sex : 0.025
Age : 0.053
Fare : 0.070
Cabin : 0.014
Embarked : 0.302
Title : 0.003
Null_Age : 0.037
group_size : 0.047
Null_Cabin : 0.048


In [ ]:
submission = pd.DataFrame({

    'PassengerId':test['PassengerId']
    ,'Survived':prediction
    
})
submission.to_csv('submission.csv',index=False)